In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [2]:
import numpy as np
import os.path as osp
from evaluator import Evaluator
from general_tools.in_out.basics import files_in_subdirs

In [4]:
class_name = 'chair'
# generative_type = 'latent_gan', 'gmm', or 'raw_gan'
generative_type = 'raw_gan'
do_jsd = False
do_mmd = True
mmd_loss = 'emd'

In [5]:
top_sample_dir = '/orions4-zfs/projects/optas/DATA/OUT/iclr/synthetic_samples/r_gan/chair_mlp_disc_4_fc_gen_raw_gan_2048_pts/'

In [6]:
if do_jsd:
    f_jsd = open(osp.join(top_sample_dir, 'JSD_Measurements.txt'), 'w')
if do_mmd:
    f_mmd = open(osp.join(top_sample_dir, 'MMD_Measurements.txt'), 'w')

In [7]:
if generative_type == 'latent_gan':
    epochs_to_check = np.hstack([np.array([1, 5, 10]), np.arange(100, 2001, 100)])
elif generative_type == 'raw_gan':
    epochs_to_check = np.hstack([np.array([1, 5, 10]), np.arange(50, 2001, 50)])

In [8]:
sample_files = []
if generative_type.endswith('gan'):    
    for epoch in epochs_to_check:
        sample_files.append(osp.join(top_sample_dir, 'epoch_%d.npz' % (epoch,) ))
elif generative_type.startswith('gmm'):
    for f in files_in_subdirs(top_sample_dir, '.npz'):
        sample_files.append(f) 
else:
    assert False

In [9]:
evaluator = Evaluator(class_name)
evaluator.load_gt_data()
if do_jsd:
    evaluator.prepare_gt_grid_variables()

In [ ]:
for sample_file in sample_files:    
    evaluator.prepare_sample_data(sample_file)
    print osp.basename(sample_file)
    if do_jsd:
        print 'JSD'
        f_jsd.write(osp.basename(sample_file) + '\n')
        evaluator.compute_jsd(f_out=f_jsd)
        f_jsd.flush()
    if do_mmd:
        print 'MMD'
        f_mmd.write(osp.basename(sample_file) + '\n')
        evaluator.compute_mmd(loss=mmd_loss, f_out=f_mmd)
        f_mmd.flush()

In [ ]:
if do_jsd:
    f_jsd.close()
if do_mmd:
    f_mmd.close()